# Caso Práctico Unidad 3: Aprendizaje Semi-Supervisado

En este caso práctico vamos a poner en práctica las técnicas de aprendizaje semi-supervisado que hemos visto en clase. 

Para este caso se han creado unos datos sintéticos que simulan un problema de clasificación binaria con 5 características. Los datos se dividen en dos datasets:
* 'supervisado_train.csv': casi 1000 muestras de entrenamiento pero únicamente 6 de ellas están etiquetadas.
* 'supervisado_test.csv': 100 muestras etiquetadas que únicamente se usarán para evaluar el rendimiento del modelo.

¿Seremos capaces de entrenar un modelo de ML con únicamente 6 muestras etiquetadas?


3. Modelo de Auto-Aprendizaje

Por último, vamos a utilizar un modelo de auto-aprendizaje y seguir los mismos pasos que en el modelo anterior.

La clase que debemos utilizar se llama `SelfTrainingClassifier`, también dentro de la librería de `sklearn.semi_supervised`. Además de poner los targets no etiquetados a -1 en este caso tenemos que enviarle el modelo base que queremos utilizar en nuestro auto-aprendizaje, prueba tanto con regresión logística como con un Support Vector Machine.



In [25]:
import pandas as pd
import numpy as np
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [26]:
# Cargar los datos
train = pd.read_csv('data/semisupervisado_train.csv')
test = pd.read_csv('data/semisupervisado_test.csv')

print(train.head())
print(test.head())

# Los valores no etiquetados se deben poner como -1
train['target'] = train['target'].fillna(-1)

# Separar las características (X) y el target (y)
X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

# Probar con Regresión Logística
modelo_base_lr = LogisticRegression(max_iter=1000)
modelo_autoaprendizaje_lr = SelfTrainingClassifier(base_estimator=modelo_base_lr, criterion='threshold', threshold=0.75)
modelo_autoaprendizaje_lr.fit(X_train, y_train)

# Probar con SVM
modelo_base_svm = SVC(probability=True)
modelo_autoaprendizaje_svm = SelfTrainingClassifier(base_estimator=modelo_base_svm, criterion='threshold', threshold=0.75)
modelo_autoaprendizaje_svm.fit(X_train, y_train)

# Evaluarcon Regresión Logística
pred_lr = modelo_autoaprendizaje_lr.predict(X_test)
print("\nResultados con Regresión Logística:")
print(classification_report(y_test, pred_lr))

# Evaluar con SVM
pred_svm = modelo_autoaprendizaje_svm.predict(X_test)
print("\nResultados con Support Vector Machine:")
print(classification_report(y_test, pred_svm))


   feature_1  feature_2  feature_3  feature_4  feature_5  target
0  -2.088601  -0.340637   1.884864  -1.392464  -1.085290     NaN
1   0.524162   1.004460  -0.679015   0.818031  -1.604752     NaN
2  -0.387311  -0.764898  -0.218967  -1.191786   0.960937     NaN
3   1.980883  -1.446289  -0.516890   0.456531   1.169348     NaN
4   3.513369  -1.237824  -0.411697   2.373793   1.115556     NaN
   feature_1  feature_2  feature_3  feature_4  feature_5  target
0  -0.126458   1.858739   0.239536   1.440887  -2.927498     1.0
1   1.768867  -1.619473  -3.260053  -1.286733   4.997077     1.0
2   2.022918  -1.976099  -1.789629  -0.990820   1.046086     1.0
3   2.143243  -2.128542   1.710229   1.140884  -1.223265     0.0
4   1.251846  -0.731237   1.437807   1.656303   0.126129     0.0

Resultados con Regresión Logística:
              precision    recall  f1-score   support

         0.0       0.48      0.42      0.44        48
         1.0       0.52      0.58      0.55        52

    accuracy       

### 1. Modelo de Regresión Logística

Entrena y evalúa un modelo normal de aprendizaje semisupervisado utilizando únicamente las 6 muestras etiquetadas disponibles en train y extrae las métricas de evaluación (accuracy, precisión, recall...) con el conjunto de test. ¿Qué te parecen los resultados?

In [27]:
# Seleccionar solo las filas que tienen una etiqueta conocida (no igual a -1)
X_train_labeled = X_train[y_train != -1]
y_train_labeled = y_train[y_train != -1]

# Crear el modelo
modelo_lr = LogisticRegression(max_iter=1000)

# Entrenar con las muestras etiquetadas
modelo_lr.fit(X_train_labeled, y_train_labeled)

# Evaluar en los datos de prueba
predictions = modelo_lr.predict(X_test)

# Mostrar métricas de evaluación
print("Resultados con Regresión Logística (solo muestras etiquetadas):")
print(classification_report(y_test, predictions))

Resultados con Regresión Logística (solo muestras etiquetadas):
              precision    recall  f1-score   support

         0.0       0.44      0.35      0.39        48
         1.0       0.49      0.58      0.53        52

    accuracy                           0.47       100
   macro avg       0.46      0.47      0.46       100
weighted avg       0.46      0.47      0.46       100



### 2. Modelo de Propagación de Etiquetas

Ahora vamos a utilizar todos los datos de entrenamiento disponibles mediante un modelo de aprendizaje supervisado, concretamente el de propagación de etiquetas.

Recuerda que para ello utilizaremos el modelo `LabelPropagation` dentro de la librería de `sklearn.semi_supervised` y que para entrenarlo debemos poner el target a -1 en aquellas muestras no etiquetadas.

Evalúa de nuevo el modelo con el conjunto de test y compara los resultados con el modelo anterior.

In [28]:
from sklearn.semi_supervised import LabelPropagation

# Crear el modelo
modelo_lp = LabelPropagation()

# Entrenar con todas las muestras (etiquetadas y no etiquetadas)
modelo_lp.fit(X_train, y_train)

# Evaluar
predictions_lp = modelo_lp.predict(X_test)

# Mostrar métricas de evaluación
print("Resultados con Propagación de Etiquetas:")
print(classification_report(y_test, predictions_lp))


Resultados con Propagación de Etiquetas:
              precision    recall  f1-score   support

         0.0       0.75      0.25      0.38        48
         1.0       0.57      0.92      0.71        52

    accuracy                           0.60       100
   macro avg       0.66      0.59      0.54       100
weighted avg       0.66      0.60      0.55       100



c:\Users\nunoc\anaconda3\Lib\site-packages\sklearn\semi_supervised\_label_propagation.py:322: ConvergenceWarning: max_iter=1000 was reached without convergence.
  warnings.warn(



### 3. Modelo de Auto-Aprendizaje

Por último, vamos a utilizar un modelo de auto-aprendizaje y seguir los mismos pasos que en el modelo anterior.

La clase que debemos utilizar se llama `SelfTrainingClassifier`, también dentro de la librería de `sklearn.semi_supervised`. Además de poner los targets no etiquetados a -1 en este caso tenemos que enviarle el modelo base que queremos utilizar en nuestro auto-aprendizaje, prueba tanto con regresión logística como con un Support Vector Machine.

In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC

# Crear y entrenar con Regresión Logística
modelo_base_lr = LogisticRegression(max_iter=1000)
modelo_autoaprendizaje_lr = SelfTrainingClassifier(base_estimator=modelo_base_lr, criterion='threshold', threshold=0.75)
modelo_autoaprendizaje_lr.fit(X_train, y_train)

# Evaluar con los datos de prueba
predictions_lr = modelo_autoaprendizaje_lr.predict(X_test)

# Mostrar métricas de evaluación para Regresión Logística
print("Resultados con Auto-Aprendizaje (Regresión Logística):")
print(classification_report(y_test, predictions_lr))

# Crear y entrenar el modelo con SVM
modelo_base_svm = SVC(probability=True)
modelo_autoaprendizaje_svm = SelfTrainingClassifier(base_estimator=modelo_base_svm, criterion='threshold', threshold=0.75)
modelo_autoaprendizaje_svm.fit(X_train, y_train)

# Evaluar con los datos de prueba
predictions_svm = modelo_autoaprendizaje_svm.predict(X_test)

# Mostrar métricas de evaluación para SVM
print("Resultados con Auto-Aprendizaje (SVM):")
print(classification_report(y_test, predictions_svm))


Resultados con Auto-Aprendizaje (Regresión Logística):
              precision    recall  f1-score   support

         0.0       0.48      0.42      0.44        48
         1.0       0.52      0.58      0.55        52

    accuracy                           0.50       100
   macro avg       0.50      0.50      0.49       100
weighted avg       0.50      0.50      0.50       100

Resultados con Auto-Aprendizaje (SVM):
              precision    recall  f1-score   support

         0.0       0.48      0.33      0.40        48
         1.0       0.52      0.67      0.59        52

    accuracy                           0.51       100
   macro avg       0.50      0.50      0.49       100
weighted avg       0.50      0.51      0.50       100

